attention layer first layer value without divided by sqrt of value dim based on model_5-1

 use trainable PE based on model_5-3
 
 we are going to find out whether ffn is necessary

In [1]:
import tensorflow as tf
import numpy as np
import utils

In [2]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [3]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEncodeInit(tf.keras.initializers.Initializer):
    def __init__(self):
        pass
    def __call__(self, shape):
        return positional_encoding(*shape)

class Attention(tf.keras.layers.Layer):
    def __init__(self,target_count,qk_dim,first=False):
        super().__init__()
        self.target_count = target_count
        self.qk_dim = qk_dim
        self.first = first
        
    def build(self,input_shape):
        if self.first:
            self.input_positional_encode = PositionalEncodeInit()(input_shape[1:])
#             self.input_positional_encode = positional_encoding(*input_shape[1:])
        self.Q = PositionalEncodeInit()((self.target_count,self.qk_dim))
#         self.Q = positional_encoding(self.target_count,self.qk_dim)
        self.K = tf.keras.layers.Dense(self.qk_dim)
        self.V = tf.keras.layers.Dense(input_shape[-1])
#         self.ff = tf.keras.Sequential([
#             tf.keras.layers.Dense(input_shape[-1]//2, activation=mish),
#             tf.keras.layers.Dense(input_shape[-1])
#         ])
        
    def call(self,value,training):
        if self.first:
#             value = value * tf.math.sqrt(tf.cast(tf.shape(value)[-1], tf.float32))
            value = value + self.input_positional_encode
        #value : (B,value_count,value_dim)
        #K: (B,value_count,qk_dim)
        V = self.V(value)
        K = self.K(value)
        #Q: (B,target_count,qk_dim)
        QK = tf.matmul(self.Q,K,transpose_b=True)
        QK = QK / tf.sqrt(tf.cast(self.qk_dim,tf.float32))
        QK = tf.keras.activations.softmax(QK,axis=-1)
        
        #QK : (B,target_count,value_count)  #value: (B,value_count,value_dim)
        output = tf.matmul(QK,V)
        #output : (B,target_count,value_dim)
#         output = self.ff(output)
        return output

In [4]:
class Conv2D(tf.keras.layers.Conv2D):
    def __init__(self,f,k,a,b,**kwdarg):
        super().__init__(
            filters=f,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [5]:
class DepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self,m,k,a,b,**kwdarg):
        super().__init__(
            depth_multiplier=m,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [6]:
class DepthControl(tf.keras.layers.Layer):
    def __init__(self,r,a,b,groups=1):
        super().__init__()
        self.r = 2**r
        self.a = a
        self.b = b
        self.groups = groups
    def build(self,input_shape):
        self.conv = Conv2D(
            f=int(input_shape[-1]*self.r),
            k=1,
            a=self.a,
            b=self.b,
            groups=self.groups
        )
    def call(self,inp,training):
        x = self.conv(inp,training=training)
        return x

In [7]:
class SubBlock(tf.keras.layers.Layer):
    def __init__(self,vision_layer,nins):
        super().__init__()
        self.vision_layer = vision_layer
        self.main = tf.keras.Sequential([DepthControl(r=r,a=mish,b=True) for r in nins]+
                                        [DepthControl(r=-r,a=mish,b=True) for r in nins[::-1]])
        self.mix = DepthControl(r=-1,a=mish,b=False)
    def call(self,inp,training):
        x1 = self.vision_layer(inp,training=training)
        x2 = self.main(x1,training=training)
        x = tf.concat([x1,x2],axis=-1)
        x = self.mix(x,training=training)
        return x

In [8]:
class GroupSubBlock(tf.keras.layers.Layer):
    def __init__(self,vision_layer,groups,nins):
        super().__init__()
        self.vision_layer = vision_layer
        self.main = tf.keras.Sequential([DepthControl(r=r,a=mish,b=True,groups=groups) for r in nins]+
                                        [DepthControl(r=-r,a=mish,b=True,groups=groups) for r in nins[::-1]])
    def call(self,inp,training):
        x1 = self.vision_layer(inp,training=training)
        x2 = self.main(x1,training=training)
        x = x1 + x2
        return x

In [9]:
class Contract(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.downsample = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid')
        self.increase = DepthControl(r=1,a=None,b=False)
    def call(self,inp,training):
        x = self.downsample(inp,training=training)
        x = self.increase(x,training=training)
        return x

In [10]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.block1 = tf.keras.Sequential([ #B,40,240,1
            SubBlock(DepthwiseConv2D(m=8,k=3,a=None,b=False),[1,1]),  #B.38,238,8
            SubBlock(Conv2D(f=8,k=3,a=mish,b=True),[1,1]), #B.36,236,8
            Contract()]) #B,18,118,16
        self.block2 = tf.keras.Sequential([
            SubBlock(Conv2D(f=16,k=3,a=mish,b=True),[1,0,0]), #B,16,116,16
            Contract()]) #B,8,58,32
        self.block3 = tf.keras.Sequential([
            SubBlock(Conv2D(f=32,k=3,a=mish,b=True),[1,0,0]), #B,6,56,32
            SubBlock(Conv2D(f=32,k=3,a=mish,b=True),[1,0,0]), #B,4,54,,32
            Contract()]) #B,2,27,64
        self.block4 = tf.keras.Sequential([
            SubBlock(Conv2D(f=64,k=2,a=mish,b=True),[-1,0,0]), #B,1,26,64
        ])
        self.block5 = tf.keras.Sequential([
            SubBlock(Conv2D(f=128,k=(1,2),a=mish,b=True),[0,0]), #B,1,25,128
            SubBlock(Conv2D(f=256,k=(1,2),a=mish,b=True),[0,0]), #B,1,24,256
#             tf.keras.layers.AveragePooling2D(pool_size=(1,2), strides=(1,2), padding='valid'),
            tf.keras.layers.Lambda(lambda x: tf.squeeze(x,axis=[1])), #B,24,256
#             Attention(22,64,True),
#             Attention(20,64),
#             Attention(18,64),
#             Attention(16,64),
#             Attention(14,64),
#             Attention(12,64),
            Attention(12,256,True),
        ])
        self.output_layer = tf.keras.Sequential([
            tf.keras.layers.Dense(128,activation=mish,use_bias=True),
            tf.keras.layers.Dense(64,activation=mish,use_bias=True),
            tf.keras.layers.Dense(36,activation=None,use_bias=False)
        ])
        
    def call(self,image,training):
        x = self.block1(image,training=training)
        x = self.block2(x,training=training)
        x = self.block3(x,training=training)
        x = self.block4(x,training=training)
        x = self.block5(x,training=training)
        x = self.output_layer(x,training=training)
        return x
    
    def predict(self,image):
        raw_predict = self(image,training=False)
        predict = tf.argmax(raw_predict,axis=-1)
        predict = predict.numpy()
        predict = [utils.token2char(p) for p in predict]
        return predict

In [11]:
model = Model()

In [12]:
import FLAGS

FLAGS.DATA.TRAIN.TABLE_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\train\overturn\cleaned_merge_table.csv'
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RATIO = 0.05
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RANDOM_STATE = 100
FLAGS.DATA.TRAIN.IMAGE_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\train\overturn\image'
FLAGS.DATA.TRAIN.TRAIN_BATCH_SIZE = 4
FLAGS.DATA.TRAIN.VAL_BATCH_SIZE = 8
FLAGS.DATA.TEST.IMAGE_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\test\overturn\sample'
FLAGS.DATA.TEST.BATCH_SIZE = 8

FLAGS.MODEL = model

FLAGS.LOSS.LABEL_SMOOTHING = 0.01

FLAGS.OPTIMIZER.TYPE = tf.keras.optimizers.Adam
FLAGS.OPTIMIZER.MAX_LEARNING_RATE = 1e-3
FLAGS.OPTIMIZER.SCHEDULE_GAMMA = -1

FLAGS.EPOCHS.TOTAL = 300
FLAGS.EPOCHS.WARMUP = 2

FLAGS.LOGGING.PATH = 'logs'
FLAGS.LOGGING.MODEL_NAME = '5-4'
FLAGS.LOGGING.TRIAL_NUMBER = 2
FLAGS.LOGGING.NOTE = 'use train.py'
FLAGS.LOGGING.SAMPLES_PER_LOG = 128
FLAGS.LOGGING.TEST_IMAGE_COLUMNS = 5

In [13]:
%run train.py

KeyboardInterrupt: 

conclusion:

maybe the existing of ffn is not necessary in this case